# Dense Retrieval

## Install Requirements

In [2]:
!pip install -rq requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for cohere from https://files.pythonhosted.org/packages/db/35/6e4927c7aeb9ea426312cacf619cbd94fa9ac3cb1002cc6b6cd066180ce2/cohere-4.27-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 407.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for fastavro==1.8.2 from https://files.pythonhosted.org/packages/5b/74/da6fd6b4a818b9ffa4912988c47e432716ab03a4b5015524aabf5a9aa519/fastavro-1.8.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 715.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 597.5 kB/s eta 0:00:0000:0100:01
  Created wheel for Annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=77683 sha256=bc4fdca9e359aa53c04cea758b0cea396b7e46665c9ff35d75029700fadeb12b
  Stored in 

# Cohere API key
Create an `.env` file and add the Cohere API key to the following key `COHERE_API_KEY`


In [3]:
import os
from dotenv import load_dotenv
#
load_dotenv(".env")
COHERE_API_KEY = os.get("COHERE_API_KEY")

NameError: name 'os' is not defined